# Global search
- following this [notebook](https://microsoft.github.io/graphrag/posts/query/notebooks/global_search_nb/)

In [ ]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

os.environ["GRAPHRAG_API_KEY"] = "QWERTY"
os.environ["GRAPHRAG_LLM_MODEL"] = "gemma2"

api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

# we're setting OpenaiApiType, but actually using ollama's gemma2 model
# running at http://localhost:11434/v1
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_base="http://localhost:11434/v1",
    api_type=OpenaiApiType.OpenAI,
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

entity_df = pd.read_parquet("./output/20240713-095103/artifacts/create_final_nodes.parquet")
report_df = pd.read_parquet("./output/20240713-095103/artifacts/create_final_community_reports.parquet")
entity_embedding_df = pd.read_parquet("./output/20240713-095103/artifacts/create_final_entities.parquet")

In [ ]:
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 105


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,100,# Gorbunov & Bogdanov's Framework on 21st Cent...,2,8.5,Gorbunov & Bogdanov's Framework on 21st Centur...,The community focuses on a critical topic – th...,This community centers around the research of ...,[{'explanation': 'Gorbunov and Bogdanov's rese...,"{\n ""title"": ""Gorbunov & Bogdanov's Framewo...",0388c716-fee7-4214-88c0-873448939f81
1,101,# Military Thought & War Definitions Community...,2,7.5,Military Thought & War Definitions Community,This community explores a significant topic wi...,This community explores various perspectives o...,[{'explanation': 'The community highlights con...,"{\n ""title"": ""Military Thought & War Defini...",a27293ff-70ed-4a92-be0e-f870dce96fd0
2,102,# Soviet Military Thought Community\n\nThis co...,2,8.5,Soviet Military Thought Community,The Soviet Union's military doctrine significa...,This community centers around the Soviet Union...,[{'explanation': 'The Soviet Union's military ...,"{\n ""title"": ""Soviet Military Thought Commu...",9207f5c2-9011-453b-a484-2a6b072559ac
3,103,# Soviet Union's View on War\n\nThis report an...,2,7.5,Soviet Union's View on War,The Soviet Union's historical impact on global...,This report analyzes the Soviet Union's perspe...,[{'explanation': 'The Soviet Union believed th...,"{\n ""title"": ""Soviet Union's View on War"",\...",499e66ef-a302-4d0b-a50a-13ad4d407a41
4,104,# Military & Political Thought on War\n\nThis ...,2,7.5,Military & Political Thought on War,The community delves into influential theories...,This community explores various perspectives o...,[{'explanation': 'Clausewitz's belief that all...,"{\n ""title"": ""Military & Political Thought ...",c51e292c-33df-4d67-aace-54e1df23a2c9


In [ ]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [ ]:
context_builder_params = {


    # use TRUE for presentation -much faster!
    "use_community_summary": True,  # False means using full community reports. True means using community short summaries.


    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 3_000,  # changed from 12_000 to 3_000
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 1000,  # changed from 2_000 to 1_000
    "temperature": 0.0,
}

In [ ]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=3_000,  # changed from 12_000 to 3_000
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,
    json_mode=False,  # I set this to False
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",
)

In [ ]:
result = await search_engine.asearch(
    # "What are the differences between Russian and Western approaches to war?"
)

print(result.response)

In [ ]:
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,104,Military & Political Thought on War,0.322981,# Military & Political Thought on War\n\nThis ...,7.5
1,32,Russian Military Doctrine & Influence,0.229814,# Russian Military Doctrine & Influence\n\nThi...,8.5
2,86,Russian Military Doctrine & Influences,0.155280,# Russian Military Doctrine & Influences\n\nTh...,8.5
3,40,Russian Information Warfare Strategies,0.080745,# Russian Information Warfare Strategies\n\nTh...,8.5
4,34,Korybko's Theory of Warfare & Color Revolutions,0.074534,# Korybko's Theory of Warfare & Color Revoluti...,7.5
...,...,...,...,...,...
66,35,Soviet Military Thought Community,0.049689,# Soviet Military Thought Community\n\nThe Sov...,7.5
67,88,Post-Russo-Georgian War Reforms in Russia,0.018634,# Post-Russo-Georgian War Reforms in Russia\n\...,7.5
68,103,Soviet Union's View on War,0.018634,# Soviet Union's View on War\n\nThis report an...,7.5
69,37,Russian Military & Warfare Perspectives,0.018634,# Russian Military & Warfare Perspectives\n\nT...,6.5


In [ ]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 13. LLM tokens: 45026


In [ ]:
# The Russian Understanding of War_ Blurring the Lines Between.pdf
questions_based_on_the_contents = (
    "What are the main themes discussed in the introduction of the book?",
    # chapter 1
    "How did Soviet military science define the cause of war?",
    "What role did politics play in the Soviet understanding of war?",
    "Who was Evgeny Messner, and what were his contributions to Soviet military thought?",
    # chapter 2
    "How did the understanding of war in Russia evolve during the 1990s?",
    "What were the major changes in Russia's military strategy in the 2010s?",
    # chapter 3
    "What are the key elements of Russia's Information Security Doctrines?",
    "Can you explain the difference between information-technical warfare and information-psychological warfare in the Russian context?",
    # chapter 4
    "How does Russia's political view on color revolutions differ from its military view?",
    "What are the key conclusions drawn in the chapter on color revolutions?",
    # conclusions
    "What are the major policy implications discussed in the conclusion of the book?"
)

In [ ]:
interactions = {}
for idx, question in enumerate(questions_based_on_the_contents, start=1):
    result = await search_engine.asearch(question)
    interactions[question] = result.response
    print(idx)

In [ ]:
for q, a in interactions.items():
    print(f'- "{q}"\n\n"""\n{a}\n"""\n\n')

- "What are the main themes discussed in the introduction of the book?"

"""
The introduction likely discusses the Soviet Union's military doctrine and its revision [Data: Reports (13)].  

The Soviet Union believed that certain types of wars were justified, specifically those aimed at defending socialist countries or supporting national liberation movements. This belief stemmed from their ideology [Data: Reports (13)]. 

"""


- "How did Soviet military science define the cause of war?"

"""
The Soviet Union believed that war was a consequence of class struggle, a central tenet of their Marxist-Leninist ideology.  This belief held that wars were inevitable outcomes of the inherent conflict between social classes. 

Furthermore, the Soviet Union viewed war as a tool for achieving revolutionary goals and spreading socialism globally [Data: Reports (13)]. They believed certain types of wars were justified, particularly those aimed at defending socialist countries or supporting national l

In [ ]:
# context sent to LLM
result.context_data

{'reports':      id                                            title  occurrence weight  \
 0   104              Military & Political Thought on War           0.322981   
 1    32            Russian Military Doctrine & Influence           0.229814   
 2    86           Russian Military Doctrine & Influences           0.155280   
 3    40           Russian Information Warfare Strategies           0.080745   
 4    34  Korybko's Theory of Warfare & Color Revolutions           0.074534   
 ..  ...                                              ...                ...   
 66   35                Soviet Military Thought Community           0.049689   
 67   88        Post-Russo-Georgian War Reforms in Russia           0.018634   
 68  103                       Soviet Union's View on War           0.018634   
 69   37          Russian Military & Warfare Perspectives           0.018634   
 70   73                        URSS Publishing Community           0.012422   
 
                           

In [ ]:
# generated context text
result.context_text

['\n\nid|title|occurrence weight|content|rank\n104|Military & Political Thought on War|0.32298136645962733|"# Military & Political Thought on War\n\nThis community explores various perspectives on war, primarily focusing on its political and strategic dimensions. Key entities include thinkers like Clausewitz, Lenin, Khrushchev, and Garthoff, who offer diverse interpretations of war\'s nature, causes, and consequences. The community highlights the evolving understanding of war in modern times, encompassing not just military force but also economic, diplomatic, technological, and ideological aspects.\n\n## War is fundamentally political.\n\nClausewitz\'s belief that all wars are inherently political in purpose is a central theme within this community. This perspective emphasizes the interconnectedness of war and politics, suggesting that conflicts arise from underlying power struggles and ideological differences. [Data: WAR (207), Clausewitz]  This idea resonates with other thinkers like

In [ ]:
question

'What are the major policy implications discussed in the conclusion of the book?'

In [ ]:
result.response

'I am sorry but I am unable to answer this question given the provided data.'